In [11]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import pyarrow as pa
import os

In [45]:
# Cargar los datos desde los archivos CSV
chile_df = pd.read_csv('data/chile.csv',  low_memory=False)
argentina_df = pd.read_csv('data/argentina.csv',  low_memory=False)
antartica_df = pd.read_csv('data/antartica.csv',  low_memory=False)

# Mostrar la cantidad de columnas y la información de cada DataFrame
dataframes = {
    'Chile': chile_df,
    'Argentina': argentina_df,
    'Antártica': antartica_df
}

for name, df in dataframes.items():
    print(f"DataFrame: {name}")
    print(f"Número de columnas: {len(df.columns)}")
    print("Información de las columnas:")
    print(df.info(5))
    print("\n")


DataFrame: Chile
Número de columnas: 146
Información de las columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9481 entries, 0 to 9480
Data columns (total 146 columns):
 #    Column               Dtype  
---   ------               -----  
 0    occurrence_no        int64  
 1    record_type          object 
 2    reid_no              float64
 3    flags                object 
 4    collection_no        int64  
 5    accepted_name        object 
 6    accepted_attr        float64
 7    accepted_rank        object 
 8    accepted_no          float64
 9    early_interval       object 
 10   late_interval        object 
 11   max_ma               float64
 12   min_ma               float64
 13   ref_author           object 
 14   ref_pubyr            int64  
 15   reference_no         int64  
 16   phylum               object 
 17   phylum_no            object 
 18   class                object 
 19   class_no             object 
 20   order                object 
 21   order_no   

In [55]:
columns = [
    'species_name',
    'occurrence_no',
    'accepted_name',
    'accepted_rank',
    'early_interval',
    'late_interval',
    'max_ma',
    'min_ma',
    'phylum',
    'class',
    'order',
    'family',
    'genus',
    'lat',
    'lng',
    'diet',
    'geogscale',
    'regionalsection',
    'life_habit'
]

df_chile_new = chile_df[columns].copy()
df_argentina_new = argentina_df[columns].copy()
df_antartica_new = antartica_df[columns].copy()

In [57]:
df_chile_new = chile_df.drop_duplicates()
df_argentina_new = argentina_df.drop_duplicates()
df_antartica_new = antartica_df.drop_duplicates()

In [53]:
df_chile_new

,occurrence_no,record_type,reid_no,flags,collection_no,accepted_name,accepted_attr,accepted_rank,accepted_no,early_interval,...,collection_dates,collection_comments,taxonomy_comments,primary_reference,authorizer_no,enterer_no,modifier_no,authorizer,enterer,modifier
0,16417,occ,NaN,NaN,1314,Leionucula,NaN,genus,16064.0,Maastrichtian,...,NaN,NaN,"history: new-508, DJ 1992, replaces old m-24 (...",D. Jablonski and D. M. Raup. 1999. Unpublished...,29,29,4,D. Jablonski,D. Jablonski,J. Alroy
1,16418,occ,NaN,NaN,1314,Nuculana,NaN,genus,16083.0,Maastrichtian,...,NaN,NaN,"history: new-508, DJ 1992, replaces old m-24 (...",D. Jablonski and D. M. Raup. 1999. Unpublished...,29,29,4,D. Jablonski,D. Jablonski,J. Alroy
2,16419,occ,NaN,NaN,1314,Nuculana,NaN,genus,16083.0,Maastrichtian,...,NaN,NaN,"history: new-508, DJ 1992, replaces old m-24 (...",D. Jablonski and D. M. Raup. 1999. Unpublished...,29,29,4,D. Jablonski,D. Jablonski,J. Alroy
3,16420,occ,NaN,NaN,1314,Yoldia,NaN,genus,16145.0,Maastrichtian,...,NaN,NaN,"history: new-508, DJ 1992, replaces old m-24 (...",D. Jablonski and D. M. Raup. 1999. Unpublished...,29,29,4,D. Jablonski,D. Jablonski,J. Alroy
4,16421,occ,NaN,NaN,1314,Neiloides,NaN,genus,16079.0,Maastrichtian,...,NaN,NaN,"history: new-508, DJ 1992, replaces old m-24 (...",D. Jablonski and D. M. Raup. 1999. Unpublished...,29,29,4,D. Jablonski,D. Jablonski,J. Alroy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9476,1659402,occ,NaN,NaN,235517,Ostrea,NaN,genus,16870.0,Turonian,...,NaN,NaN,NaN,"O. Wilckens. 1907. Die Lamellibrachiaten, Gas...",166,1010,0,M. Clapham,J. Fearon,NaN
9477,1659403,occ,NaN,NaN,235517,Cucullaea antarctica,NaN,species,493883.0,Turonian,...,NaN,NaN,NaN,"O. Wilckens. 1907. Die Lamellibrachiaten, Gas...",166,1010,0,M. Clapham,J. Fearon,NaN
9478,1659404,occ,NaN,NaN,235517,Nucula,NaN,genus,16082.0,Turonian,...,NaN,NaN,NaN,"O. Wilckens. 1907. Die Lamellibrachiaten, Gas...",166,1010,0,M. Clapham,J. Fearon,NaN
9479,1660104,occ,NaN,NaN,235648,Micrauchenia saladensis,NaN,species,494711.0,Tortonian,...,2005,Holotype is stored in the Vertebrate Paleontol...,NaN,"H. P. Püschel, J. Alarcón-Muñoz, and S. Sot...",18,1218,0,M. Uhen,W. Worrell,NaN
